In [5]:
# ============================================================================
# EXTRAÇÃO DE VOCAIS - Amy Winehouse
# Google Colab - Código Simplificado
# ============================================================================

# CELL 1: Instalação de dependências
# ----------------------------------------------------------------------------
!pip install -q yt-dlp demucs librosa soundfile
!apt-get install -q -y ffmpeg

# CELL 2: Configurar Google Drive
# ----------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

import os

# Diretório para salvar os áudios
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/AudioAnalysis/AmyWinehouse'
os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)
print(f"✓ Diretório criado: {DRIVE_OUTPUT_PATH}")

# Diretório temporário
TEMP_DIR = "/content/temp_audio"
os.makedirs(TEMP_DIR, exist_ok=True)

# CELL 3: Imports
# ----------------------------------------------------------------------------
import subprocess
import shutil
import torch
from demucs.pretrained import get_model
from demucs.apply import apply_model
from demucs.audio import AudioFile, save_audio

# CELL 4: Função para baixar música do YouTube
# ----------------------------------------------------------------------------
def download_audio(url, output_path):
    """
    Baixa áudio do YouTube em alta qualidade
    """
    print(f"📥 Baixando áudio de: {url}")

    os.makedirs(output_path, exist_ok=True)

    try:
        ytdlp_path = shutil.which('yt-dlp')

        if not ytdlp_path:
            print("❌ yt-dlp não encontrado. Instalando...")
            subprocess.run(['pip', 'install', '-U', 'yt-dlp'], check=True)
            ytdlp_path = shutil.which('yt-dlp')

        output_template = os.path.join(output_path, 'audio.%(ext)s')

        cmd = [
            ytdlp_path,
            '-x',  # Extrair áudio
            '--audio-format', 'wav',
            '--audio-quality', '0',  # Melhor qualidade
            '-o', output_template,
            '--no-playlist',
            '--no-check-certificate',
            '--prefer-ffmpeg',
            url
        ]

        print("Executando download...")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)

        if result.returncode == 0:
            # Procurar arquivo .wav baixado
            wav_files = [f for f in os.listdir(output_path) if f.endswith('.wav')]
            if wav_files:
                audio_file = os.path.join(output_path, wav_files[0])
                file_size = os.path.getsize(audio_file) / (1024 * 1024)
                print(f"✓ Download concluído: {file_size:.2f} MB")
                return audio_file

        print(f"❌ Erro no download")
        print(f"STDERR: {result.stderr}")
        return None

    except subprocess.TimeoutExpired:
        print("❌ Timeout: Download demorou muito")
        return None
    except Exception as e:
        print(f"❌ Erro ao baixar: {e}")
        return None

# CELL 5: Função para separar vocais usando Demucs
# ----------------------------------------------------------------------------
def separate_vocals(audio_path, output_dir):
    """
    Separa a voz do acompanhamento musical usando Demucs
    """
    if not audio_path or not os.path.exists(audio_path):
        print(f"❌ Arquivo de áudio inválido: {audio_path}")
        return None

    print("🎵 Separando voz do acompanhamento com Demucs...")
    print("   (Pode demorar 3-5 minutos)")

    try:
        # Criar diretório de saída
        os.makedirs(output_dir, exist_ok=True)

        # Determinar dispositivo (CPU ou GPU)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"   Dispositivo: {device}")

        # Carregar modelo Demucs
        print("   Carregando modelo...")
        model = get_model('htdemucs_ft')
        model.to(device)
        model.eval()

        # Carregar áudio
        print("   Processando áudio...")
        wav = AudioFile(audio_path).read(
            streams=0,
            samplerate=model.samplerate,
            channels=model.audio_channels
        )

        # Normalizar
        ref = wav.mean(0)
        wav = (wav - ref.mean()) / ref.std()

        # Mover para dispositivo
        wav = wav.to(device).unsqueeze(0)

        # Separar stems
        print("   Separando vocais...")
        with torch.no_grad():
            sources = apply_model(
                model, wav,
                device=device,
                shifts=1,
                split=True,
                overlap=0.25
            )

        # Extrair vocais (índice 3: vocals)
        vocals = sources[0, 3]

        # Desnormalizar
        vocals = vocals * ref.std() + ref.mean()

        # Salvar arquivo de vocais
        vocals_output = os.path.join(output_dir, 'vocals.wav')
        save_audio(
            vocals.cpu(),
            vocals_output,
            samplerate=model.samplerate
        )

        if os.path.exists(vocals_output):
            file_size = os.path.getsize(vocals_output) / (1024 * 1024)
            print(f"✓ Vocais separados: {file_size:.2f} MB")
            return vocals_output
        else:
            print("❌ Erro ao salvar vocais")
            return None

    except Exception as e:
        print(f"❌ Erro no Demucs: {e}")
        import traceback
        traceback.print_exc()
        return None

# CELL 6: Pipeline completo
# ----------------------------------------------------------------------------
def process_audio(youtube_url, track_name):
    """
    Pipeline completo: Download -> Separação -> Salvar
    """
    print("\n" + "="*70)
    print(f"🎵 PROCESSANDO: {track_name}")
    print("="*70 + "\n")

    # 1. Download do YouTube
    print("📥 Etapa 1/3: Download")
    audio_file = download_audio(youtube_url, TEMP_DIR)
    if not audio_file:
        print("❌ Falha no download")
        return None

    # 2. Separação de vocais
    print("\n🎵 Etapa 2/3: Separação de vocais")
    vocals_file = separate_vocals(audio_file, TEMP_DIR)
    if not vocals_file:
        print("❌ Falha na separação")
        return None

    # 3. Salvar no Google Drive
    print("\n💾 Etapa 3/3: Salvando no Google Drive")
    filename = f"{track_name.replace(' ', '_')}_vocals.wav"
    output_path = os.path.join(DRIVE_OUTPUT_PATH, filename)

    shutil.copy2(vocals_file, output_path)

    file_size = os.path.getsize(output_path) / (1024 * 1024)
    print(f"✓ Arquivo salvo: {output_path}")
    print(f"✓ Tamanho: {file_size:.2f} MB")

    # Limpar arquivos temporários
    try:
        shutil.rmtree(TEMP_DIR)
        os.makedirs(TEMP_DIR, exist_ok=True)
    except:
        pass

    print("\n" + "="*70)
    print("✅ PROCESSAMENTO CONCLUÍDO!")
    print("="*70)

    return output_path

# CELL 7: EXECUTAR - Back to Black
# ----------------------------------------------------------------------------
# Amy Winehouse - Back to Black
YOUTUBE_URL = "https://www.youtube.com/watch?v=TJAfLE39ZZ8"
TRACK = "Back_to_Black"

result = process_audio(YOUTUBE_URL, TRACK)

if result:
    print(f"\n🎵 Vocais salvos em: {result}")
else:
    print("\n❌ Erro no processamento")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Diretório criado: /content/drive/MyDrive/AudioAnalysis/AmyWinehouse

🎵 PROCESSANDO: Back_to_Black

📥 Etapa 1/3: Download
📥 Baixando áudio de: https://www.youtube.com/watch?v=TJAfLE39ZZ8
Executando download...
✓ Download concluído: 43.95 MB

🎵 Etapa 2/3:

100%|██████████| 80.2M/80.2M [00:00<00:00, 95.1MB/s]


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/d12395a8-e57c48e6.th" to /root/.cache/torch/hub/checkpoints/d12395a8-e57c48e6.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 97.5MB/s]


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/92cfc3b6-ef3bcb9c.th" to /root/.cache/torch/hub/checkpoints/92cfc3b6-ef3bcb9c.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 90.5MB/s]


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/04573f0d-f3cf25b2.th" to /root/.cache/torch/hub/checkpoints/04573f0d-f3cf25b2.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 104MB/s]


   Processando áudio...
   Separando vocais...
✓ Vocais separados: 40.38 MB

💾 Etapa 3/3: Salvando no Google Drive
✓ Arquivo salvo: /content/drive/MyDrive/AudioAnalysis/AmyWinehouse/Back_to_Black_vocals.wav
✓ Tamanho: 40.38 MB

✅ PROCESSAMENTO CONCLUÍDO!

🎵 Vocais salvos em: /content/drive/MyDrive/AudioAnalysis/AmyWinehouse/Back_to_Black_vocals.wav


In [6]:
"""
Sistema Completo de Análise Vocal para Replicação de Cantores
Código Python para Google Colab - VERSÃO MELHORADA
Extrai letras e analisa TODAS as características vocais com relatório prático

INSTRUÇÕES:
1. Copie todo este código
2. Cole em uma célula do Google Colab
3. Ajuste o AUDIO_PATH com o caminho do seu arquivo
4. Ajuste SEGMENTACAO_TEMPO para controlar granularidade da análise
5. Execute a célula
"""

# ============================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# ============================================================================
print("📦 Instalando dependências...")

import os
import sys
import subprocess

def instalar_dependencias():
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)

    pacotes = [
        "torch",
        "librosa",
        "pydub",
        "SpeechRecognition",
        "praat-parselmouth",
        "scipy",
        "openai-whisper"
    ]

    for pacote in pacotes:
        print(f"→ Instalando {pacote}...")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pacote], check=True)

    subprocess.run(["apt-get", "install", "-y", "-qq", "ffmpeg"], check=True)
    print("✓ Dependências instaladas com sucesso!\n")

try:
    import whisper
    import librosa
    import parselmouth
except ImportError:
    instalar_dependencias()
    import whisper
    import librosa
    import parselmouth

print("✅ Ambiente pronto para análise vocal!")

# ============================================================================
# IMPORTAR BIBLIOTECAS
# ============================================================================
import whisper
import librosa
import numpy as np
from google.colab import drive
import torch
import parselmouth
from parselmouth.praat import call
from scipy import signal
from scipy.stats import kurtosis, skew
import json
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas!")

# ============================================================================
# MONTAR GOOGLE DRIVE
# ============================================================================
print("\n📁 Montando Google Drive...")
drive.mount('/content/drive')
print("✓ Drive montado!")

# ============================================================================
# CONFIGURAÇÕES - AJUSTE AQUI!
# ============================================================================
print("\n⚙️ Configurações:")
print("="*80)

# CAMINHO DO ARQUIVO DE ÁUDIO
AUDIO_PATH = '/content/drive/MyDrive/AudioAnalysis/AmyWinehouse/Back_to_Black_vocals.wav'

# MODELO WHISPER: 'tiny', 'base', 'small', 'medium', 'large'
MODEL_SIZE = 'medium'

# ⭐ NOVA CONFIGURAÇÃO: SEGMENTAÇÃO DE TEMPO ⭐
# Define o tamanho máximo de cada segmento para análise detalhada
# Valores menores = análise mais granular (detecta variações rápidas)
# Valores maiores = análise mais geral (média de características)
SEGMENTACAO_TEMPO = 3.0  # segundos
# Exemplos:
#   1.0 = análise palavra por palavra (muito detalhado)
#   3.0 = análise por frase curta (recomendado)
#   5.0 = análise por frase longa (mais geral)
#   10.0 = análise por seção musical

print(f"✓ Arquivo: {AUDIO_PATH}")
print(f"✓ Modelo Whisper: {MODEL_SIZE}")
print(f"✓ Segmentação de tempo: {SEGMENTACAO_TEMPO}s (cada segmento será analisado separadamente)")

if os.path.exists(AUDIO_PATH):
    print(f"✓ Arquivo encontrado!")
    print(f"✓ Tamanho: {os.path.getsize(AUDIO_PATH) / (1024*1024):.2f} MB")
else:
    print(f"❌ ERRO: Arquivo não encontrado!")
    print(f"   Ajuste o AUDIO_PATH no código.")

print("="*80)

# ============================================================================
# CLASSES DE ANÁLISE (mantidas do código original)
# ============================================================================

class AnalisadorPitch:
    """Analisa F0, vibrato e características fundamentais de frequência"""

    def __init__(self):
        self.hop_length = 512

    def extrair_f0(self, audio, sr):
        """Extrai frequência fundamental (F0) usando YIN algorithm"""
        f0, voiced_flag, voiced_probs = librosa.pyin(
            audio,
            fmin=librosa.note_to_hz('C2'),
            fmax=librosa.note_to_hz('C7'),
            sr=sr
        )
        return f0, voiced_flag, voiced_probs

    def analisar_vibrato(self, f0, sr):
        """Analisa características do vibrato: taxa, extensão e regularidade"""
        f0_clean = f0[~np.isnan(f0)]

        if len(f0_clean) < 10:
            return {
                'taxa_hz': 0,
                'extensao_cents': 0,
                'regularidade': 0,
                'presente': False
            }

        fft = np.fft.fft(f0_clean - np.mean(f0_clean))
        freqs = np.fft.fftfreq(len(f0_clean), 1/sr * self.hop_length)

        mask = (freqs > 3) & (freqs < 10)
        if np.sum(mask) > 0:
            idx_max = np.argmax(np.abs(fft[mask]))
            taxa_vibrato = freqs[mask][idx_max]
        else:
            taxa_vibrato = 0

        if len(f0_clean) > 0:
            extensao = np.std(f0_clean) / np.mean(f0_clean) * 1200
        else:
            extensao = 0

        regularidade = 1 / (1 + np.std(np.diff(f0_clean)))

        return {
            'taxa_hz': float(taxa_vibrato),
            'extensao_cents': float(extensao),
            'regularidade': float(regularidade),
            'presente': taxa_vibrato > 3 and extensao > 20
        }

    def estatisticas_f0(self, f0):
        """Estatísticas gerais do F0"""
        f0_clean = f0[~np.isnan(f0)]

        if len(f0_clean) == 0:
            return {'media': 0, 'mediana': 0, 'std': 0, 'min': 0, 'max': 0, 'range': 0}

        return {
            'media': float(np.mean(f0_clean)),
            'mediana': float(np.median(f0_clean)),
            'std': float(np.std(f0_clean)),
            'min': float(np.min(f0_clean)),
            'max': float(np.max(f0_clean)),
            'range': float(np.max(f0_clean) - np.min(f0_clean))
        }

class AnalisadorEspectral:
    """Analisa formantes, MFCC e envelope espectral"""

    def extrair_formantes(self, audio, sr, n_formantes=5):
        """Extrai formantes F1-F5 usando Praat"""
        try:
            snd = parselmouth.Sound(audio, sampling_frequency=sr)
            formants = call(snd, "To Formant (burg)", 0.0, n_formantes, 5500, 0.025, 50)

            formantes_valores = []
            for i in range(1, n_formantes + 1):
                formante_array = []
                for t in np.arange(0, snd.duration, 0.01):
                    try:
                        f = call(formants, "Get value at time", i, t, 'Hertz', 'Linear')
                        if not np.isnan(f):
                            formante_array.append(f)
                    except:
                        pass

                if formante_array:
                    formantes_valores.append({
                        f'F{i}_media': float(np.mean(formante_array)),
                        f'F{i}_std': float(np.std(formante_array)),
                        f'F{i}_min': float(np.min(formante_array)),
                        f'F{i}_max': float(np.max(formante_array))
                    })

            return formantes_valores
        except:
            return [{}] * n_formantes

    def extrair_mfcc(self, audio, sr, n_mfcc=13):
        """Extrai coeficientes MFCC"""
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)

        mfcc_stats = {}
        for i in range(n_mfcc):
            mfcc_stats[f'mfcc_{i+1}_media'] = float(np.mean(mfccs[i]))
            mfcc_stats[f'mfcc_{i+1}_std'] = float(np.std(mfccs[i]))

        return mfcc_stats

    def analisar_envelope_espectral(self, audio, sr):
        """Analisa o envelope espectral"""
        stft = np.abs(librosa.stft(audio))
        envelope = np.mean(stft, axis=1)

        centroide = librosa.feature.spectral_centroid(y=audio, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr, roll_percent=0.85)
        bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)

        return {
            'centroide_media': float(np.mean(centroide)),
            'centroide_std': float(np.std(centroide)),
            'rolloff_media': float(np.mean(rolloff)),
            'rolloff_std': float(np.std(rolloff)),
            'bandwidth_media': float(np.mean(bandwidth)),
            'bandwidth_std': float(np.std(bandwidth))
        }

class AnalisadorTemporal:
    """Analisa ADSR, timing, jitter e shimmer"""

    def analisar_adsr(self, audio, sr):
        """Analisa envelope ADSR"""
        envelope = np.abs(librosa.onset.onset_strength(y=audio, sr=sr))

        if len(envelope) == 0:
            return {'attack_time': 0, 'decay_time': 0, 'sustain_level': 0, 'release_time': 0}

        peak_idx = np.argmax(envelope)
        attack_time = peak_idx / sr * 512

        if peak_idx < len(envelope) - 1:
            sustain_level = np.mean(envelope[peak_idx:])
        else:
            sustain_level = envelope[peak_idx]

        return {
            'attack_time': float(attack_time),
            'sustain_level': float(sustain_level),
            'envelope_std': float(np.std(envelope))
        }

    def analisar_jitter_shimmer(self, audio, sr):
        """Analisa jitter e shimmer"""
        try:
            snd = parselmouth.Sound(audio, sampling_frequency=sr)
            point_process = call(snd, "To PointProcess (periodic, cc)", 75, 600)
            jitter_local = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
            shimmer_local = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

            return {
                'jitter': float(jitter_local) if not np.isnan(jitter_local) else 0,
                'shimmer': float(shimmer_local) if not np.isnan(shimmer_local) else 0
            }
        except:
            return {'jitter': 0, 'shimmer': 0}

class AnalisadorQualidade:
    """Analisa HNR, breathiness, brilho e tensão vocal"""

    def calcular_hnr(self, audio, sr):
        """Calcula Harmonics-to-Noise Ratio"""
        try:
            snd = parselmouth.Sound(audio, sampling_frequency=sr)
            harmonicity = call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
            hnr = call(harmonicity, "Get mean", 0, 0)
            return float(hnr) if not np.isnan(hnr) else 0
        except:
            return 0

    def analisar_breathiness(self, audio, sr):
        """Analisa soprosidade"""
        stft = np.abs(librosa.stft(audio))
        freqs = librosa.fft_frequencies(sr=sr)

        mask_low = freqs < 1000
        energia_baixa = np.sum(stft[mask_low])

        mask_high = freqs > 4000
        energia_alta = np.sum(stft[mask_high])

        if energia_baixa > 0:
            breathiness = energia_alta / energia_baixa
        else:
            breathiness = 0

        return {
            'breathiness_ratio': float(breathiness),
            'nivel': 'alto' if breathiness > 0.3 else 'medio' if breathiness > 0.1 else 'baixo'
        }

    def analisar_brilho_espectral(self, audio, sr):
        """Analisa brilho"""
        spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
        brilho_medio = np.mean(spectral_centroid)

        return {
            'brilho_hz': float(brilho_medio),
            'nivel': 'brilhante' if brilho_medio > 2000 else 'medio' if brilho_medio > 1000 else 'escuro'
        }

    def analisar_tensao_vocal(self, audio, sr):
        """Analisa tensão vocal"""
        stft = np.abs(librosa.stft(audio))
        freqs = librosa.fft_frequencies(sr=sr)

        mask_tensao = (freqs > 2000) & (freqs < 4000)
        energia_tensao = np.sum(stft[mask_tensao])
        energia_total = np.sum(stft)

        if energia_total > 0:
            ratio_tensao = energia_tensao / energia_total
        else:
            ratio_tensao = 0

        return {
            'tensao_ratio': float(ratio_tensao),
            'nivel': 'alta' if ratio_tensao > 0.3 else 'media' if ratio_tensao > 0.15 else 'baixa'
        }

class AnalisadorDinamico:
    """Analisa amplitude, volume e transições de registro"""

    def analisar_dinamica(self, audio, sr):
        """Analisa variações de amplitude/volume"""
        rms = librosa.feature.rms(y=audio)[0]
        db = librosa.amplitude_to_db(rms, ref=np.max)

        return {
            'rms_media': float(np.mean(rms)),
            'rms_std': float(np.std(rms)),
            'db_media': float(np.mean(db)),
            'db_std': float(np.std(db)),
            'db_min': float(np.min(db)),
            'db_max': float(np.max(db)),
            'range_dinamico': float(np.max(db) - np.min(db))
        }

    def detectar_transicoes_registro(self, f0):
        """Detecta transições entre registros vocais"""
        f0_clean = f0[~np.isnan(f0)]

        if len(f0_clean) < 10:
            return {'transicoes': [], 'numero_transicoes': 0}

        diff = np.diff(f0_clean)
        threshold = np.std(diff) * 2
        transicoes = np.where(np.abs(diff) > threshold)[0]

        return {
            'numero_transicoes': len(transicoes),
            'transicoes': transicoes.tolist()[:10]
        }

class AnalisadorProsodico:
    """Analisa inflexões, portamentos e articulação"""

    def analisar_portamentos(self, f0, sr):
        """Detecta deslizamentos entre notas"""
        f0_clean = f0[~np.isnan(f0)]

        if len(f0_clean) < 10:
            return {'portamentos_detectados': 0, 'duracao_media': 0}

        diff = np.diff(f0_clean)
        gradientes = np.gradient(diff)

        portamentos = []
        in_portamento = False
        duracao_atual = 0

        for i, g in enumerate(gradientes):
            if abs(g) > 1 and abs(g) < 10:
                if not in_portamento:
                    in_portamento = True
                    duracao_atual = 1
                else:
                    duracao_atual += 1
            else:
                if in_portamento and duracao_atual > 5:
                    portamentos.append(duracao_atual)
                in_portamento = False
                duracao_atual = 0

        if portamentos:
            duracao_media = np.mean(portamentos) * (512 / sr)
        else:
            duracao_media = 0

        return {
            'portamentos_detectados': len(portamentos),
            'duracao_media_segundos': float(duracao_media)
        }

    def analisar_articulacao(self, audio, sr):
        """Analisa padrões de articulação"""
        onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
        onsets = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)

        if len(onsets) > 1:
            intervalos = np.diff(onsets) * (512 / sr)
            taxa_articulacao = 1 / np.mean(intervalos) if np.mean(intervalos) > 0 else 0
        else:
            taxa_articulacao = 0

        return {
            'onsets_detectados': len(onsets),
            'taxa_articulacao_por_segundo': float(taxa_articulacao)
        }

class AnalisadorRessonancia:
    """Analisa ressonância peitoral, mista, cabeça, nasal e falsete"""

    def __init__(self):
        self.ranges_ressonancia = {
            'peitoral': (80, 350),
            'mista': (350, 600),
            'cabeca': (600, 1500),
            'nasal': (1500, 3000),
            'falsete': (800, 2000)
        }

    def analisar_frequencia_dominante(self, audio, sr, start_time, end_time):
        """Analisa frequência dominante em um segmento"""
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)

        start_sample = max(0, start_sample)
        end_sample = min(len(audio), end_sample)

        segmento = audio[start_sample:end_sample]

        if len(segmento) < 100:
            return None, None

        try:
            fft = np.fft.fft(segmento)
            magnitude = np.abs(fft)
            freqs = np.fft.fftfreq(len(segmento), 1/sr)

            idx_pos = freqs > 0
            freqs = freqs[idx_pos]
            magnitude = magnitude[idx_pos]

            if len(freqs) == 0 or len(magnitude) == 0:
                return None, None

            freq_dominante = freqs[np.argmax(magnitude)]

            energia_por_faixa = {}
            for nome, (freq_min, freq_max) in self.ranges_ressonancia.items():
                mask = (freqs >= freq_min) & (freqs <= freq_max)
                energia_por_faixa[nome] = np.sum(magnitude[mask])

            return freq_dominante, energia_por_faixa

        except:
            return None, None

    def determinar_ressonancia(self, freq_dominante, energia_por_faixa):
        """Determina tipo de ressonância predominante"""
        if freq_dominante is None or energia_por_faixa is None:
            return "normal"

        energia_total = sum(energia_por_faixa.values())
        if energia_total == 0:
            return "normal"

        ressonancia_principal = max(energia_por_faixa, key=energia_por_faixa.get)
        percentual = energia_por_faixa[ressonancia_principal] / energia_total

        if percentual > 0.4:
            return ressonancia_principal
        else:
            return "mista"

    def obter_marcacao(self, ressonancia):
        """Retorna marcação para o tipo de ressonância"""
        marcacoes = {
            'peitoral': '[PEITO]',
            'mista': '[MISTA]',
            'cabeca': '[CABEÇA]',
            'nasal': '[NASAL]',
            'falsete': '[FALSETE]',
            'normal': ''
        }
        return marcacoes.get(ressonancia, '')

class AnalisadorVocalCompleto:
    """Integra todos os analisadores"""

    def __init__(self):
        self.analisador_pitch = AnalisadorPitch()
        self.analisador_espectral = AnalisadorEspectral()
        self.analisador_temporal = AnalisadorTemporal()
        self.analisador_qualidade = AnalisadorQualidade()
        self.analisador_dinamico = AnalisadorDinamico()
        self.analisador_prosodico = AnalisadorProsodico()
        self.analisador_ressonancia = AnalisadorRessonancia()

    def analisar_segmento_completo(self, audio, sr, start_time, end_time):
        """Analisa todas as características de um segmento"""
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        segmento = audio[start_sample:end_sample]

        if len(segmento) < 100:
            return {}

        resultado = {}

        f0, voiced_flag, voiced_probs = self.analisador_pitch.extrair_f0(segmento, sr)
        resultado['pitch'] = self.analisador_pitch.estatisticas_f0(f0)
        resultado['vibrato'] = self.analisador_pitch.analisar_vibrato(f0, sr)

        resultado['formantes'] = self.analisador_espectral.extrair_formantes(segmento, sr)
        resultado['mfcc'] = self.analisador_espectral.extrair_mfcc(segmento, sr)
        resultado['envelope_espectral'] = self.analisador_espectral.analisar_envelope_espectral(segmento, sr)

        resultado['adsr'] = self.analisador_temporal.analisar_adsr(segmento, sr)
        resultado['jitter_shimmer'] = self.analisador_temporal.analisar_jitter_shimmer(segmento, sr)

        resultado['hnr'] = self.analisador_qualidade.calcular_hnr(segmento, sr)
        resultado['breathiness'] = self.analisador_qualidade.analisar_breathiness(segmento, sr)
        resultado['brilho'] = self.analisador_qualidade.analisar_brilho_espectral(segmento, sr)
        resultado['tensao'] = self.analisador_qualidade.analisar_tensao_vocal(segmento, sr)

        resultado['dinamica'] = self.analisador_dinamico.analisar_dinamica(segmento, sr)
        resultado['transicoes_registro'] = self.analisador_dinamico.detectar_transicoes_registro(f0)

        resultado['portamentos'] = self.analisador_prosodico.analisar_portamentos(f0, sr)
        resultado['articulacao'] = self.analisador_prosodico.analisar_articulacao(segmento, sr)

        freq_dom, energia = self.analisador_ressonancia.analisar_frequencia_dominante(audio, sr, start_time, end_time)
        ressonancia = self.analisador_ressonancia.determinar_ressonancia(freq_dom, energia)
        resultado['ressonancia'] = {
            'tipo': ressonancia,
            'freq_dominante': float(freq_dom) if freq_dom else 0,
            'marcacao': self.analisador_ressonancia.obter_marcacao(ressonancia)
        }

        return resultado

print("✓ Classes definidas!")

# ============================================================================
# FUNÇÃO: SEGMENTAR ÁUDIO POR TEMPO
# ============================================================================
def segmentar_por_tempo(segments_whisper, max_duracao):
    """
    Divide segmentos longos em pedaços menores baseado em max_duracao
    """
    segmentos_refinados = []

    for seg in segments_whisper:
        start = seg['start']
        end = seg['end']
        texto = seg['text'].strip()
        duracao = end - start

        if duracao <= max_duracao:
            # Segmento já é pequeno o suficiente
            segmentos_refinados.append(seg)
        else:
            # Dividir em subsegmentos
            num_subseg = int(np.ceil(duracao / max_duracao))
            duracao_subseg = duracao / num_subseg

            # Dividir texto proporcionalmente
            palavras = texto.split()
            palavras_por_subseg = max(1, len(palavras) // num_subseg)

            for i in range(num_subseg):
                subseg_start = start + (i * duracao_subseg)
                subseg_end = min(end, start + ((i + 1) * duracao_subseg))

                # Pegar palavras correspondentes
                idx_inicio = i * palavras_por_subseg
                idx_fim = (i + 1) * palavras_por_subseg if i < num_subseg - 1 else len(palavras)
                subseg_texto = ' '.join(palavras[idx_inicio:idx_fim])

                segmentos_refinados.append({
                    'start': subseg_start,
                    'end': subseg_end,
                    'text': subseg_texto
                })

    return segmentos_refinados

# ============================================================================
# FUNÇÃO: GERAR RELATÓRIO PRÁTICO
# ============================================================================
def gerar_relatorio_pratico(analise_global):
    """Gera relatório orientado à AÇÃO PRÁTICA"""

    rel = []

    rel.append("=" * 100)
    rel.append("GUIA PRÁTICO DE REPLICAÇÃO VOCAL - O QUE FAZER PARA IMITAR ESTA VOZ")
    rel.append("=" * 100)
    rel.append("")

    # 1. ALTURA DA VOZ
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 1. ALTURA DA VOZ (PITCH/TOM) - Qual nota cantar" + " " * 53 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    f0_media = analise_global['pitch']['media']
    f0_min = analise_global['pitch']['min']
    f0_max = analise_global['pitch']['max']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • F0 Médio: {f0_media:.1f} Hz")
    rel.append(f"   • Range: {f0_min:.1f} Hz a {f0_max:.1f} Hz")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")
    if f0_media < 150:
        rel.append(f"   ✓ Esta é uma VOZ GRAVE (Baixo/Contralto)")
        rel.append(f"   ✓ Cante na região de D3-F3 (Dó3 a Fá3)")
        rel.append(f"   ✓ Use MAIS ressonância no PEITO")
        rel.append(f"   ✓ Mantenha o som 'para baixo' e 'grosso'")
    elif f0_media < 250:
        rel.append(f"   ✓ Esta é uma VOZ MÉDIA (Barítono/Mezzo)")
        rel.append(f"   ✓ Cante na região de A3-C4 (Lá3 a Dó4)")
        rel.append(f"   ✓ Use registro MISTO (meio peito, meio cabeça)")
        rel.append(f"   ✓ Balance ressonância entre peito e cabeça")
    elif f0_media < 400:
        rel.append(f"   ✓ Esta é uma VOZ AGUDA (Tenor/Soprano)")
        rel.append(f"   ✓ Cante na região de D4-F4 (Dó4 a Fá4)")
        rel.append(f"   ✓ Use MAIS voz de CABEÇA/MISTA")
        rel.append(f"   ✓ Leve o som 'para cima' e 'para frente'")
    else:
        rel.append(f"   ✓ Esta é uma VOZ MUITO AGUDA")
        rel.append(f"   ✓ Cante acima de F4 (Fá4)")
        rel.append(f"   ✓ Use FALSETE ou voz de cabeça pura")
        rel.append(f"   ✓ Som leve e aéreo")

    rel.append("")
    rel.append("💡 COMO PRATICAR:")
    rel.append("   1. Use um afinador de celular (apps: Tuner, Pano Tuner)")
    rel.append(f"   2. Cante uma nota sustentada tentando manter {f0_media:.0f} Hz")
    rel.append("   3. Ajuste até o afinador mostrar a frequência correta")
    rel.append("")
    rel.append("")

    # 2. VIBRATO
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 2. VIBRATO - A 'tremidinha' na voz" + " " * 63 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    vibrato = analise_global['vibrato']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Presente: {'SIM' if vibrato['presente'] else 'NÃO'}")
    if vibrato['presente']:
        rel.append(f"   • Taxa: {vibrato['taxa_hz']:.1f} Hz (oscilações por segundo)")
        rel.append(f"   • Extensão: {vibrato['extensao_cents']:.1f} cents")
    rel.append("")

    if vibrato['presente']:
        if vibrato['taxa_hz'] < 4:
            velocidade = "LENTO"
            descricao = "ondulações lentas e amplas"
        elif vibrato['taxa_hz'] < 6:
            velocidade = "MÉDIO"
            descricao = "oscilação natural e equilibrada"
        else:
            velocidade = "RÁPIDO"
            descricao = "tremor rápido e intenso"

        rel.append(f"🎯 O QUE FAZER:")
        rel.append(f"   ✓ Vibrato {velocidade}: {descricao}")
        rel.append(f"   ✓ Faça cerca de {vibrato['taxa_hz']:.0f} oscilações por segundo")

        if vibrato['extensao_cents'] < 50:
            rel.append(f"   ✓ Vibrato SUTIL: variação pequena, quase imperceptível")
        elif vibrato['extensao_cents'] < 100:
            rel.append(f"   ✓ Vibrato MODERADO: variação natural")
        else:
            rel.append(f"   ✓ Vibrato INTENSO: variação grande e expressiva")

        rel.append("")
        rel.append("💡 COMO PRATICAR:")
        rel.append("   1. Cante uma nota longa e sustentada")
        rel.append(f"   2. Faça pequenas oscilações no tom, {vibrato['taxa_hz']:.0f} vezes por segundo")
        rel.append("   3. Use o diafragma: empurre levemente a barriga para dentro/fora no ritmo")
        rel.append("   4. OU relaxe a garganta e deixe oscilar naturalmente")
    else:
        rel.append(f"🎯 O QUE FAZER:")
        rel.append(f"   ✓ NÃO use vibrato, ou use muito sutil")
        rel.append(f"   ✓ Mantenha notas retas e firmes")
        rel.append("")
        rel.append("💡 COMO PRATICAR:")
        rel.append("   1. Cante notas longas sem oscilar")
        rel.append("   2. Mantenha o tom fixo, como uma flauta")

    rel.append("")
    rel.append("")

    # 3. FORMANTES
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 3. FORMANTES - Como moldar a 'COR' da voz (timbre único)" + " " * 42 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    rel.append(f"📊 DADOS TÉCNICOS:")
    if analise_global['formantes']:
        for i, formante in enumerate(analise_global['formantes'][:5], 1):
            if formante:
                media = formante.get(f'F{i}_media', 0)
                rel.append(f"   • F{i}: {media:.0f} Hz")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    # F1
    if analise_global['formantes'] and analise_global['formantes'][0]:
        f1 = analise_global['formantes'][0].get('F1_media', 0)
        rel.append(f"   F1 ({f1:.0f} Hz) - ABERTURA DA BOCA:")
        if f1 < 500:
            rel.append("      ✓ Boca POUCO aberta, mandíbula relaxada")
            rel.append("      ✓ Vogais mais fechadas (como 'i', 'u')")
        elif f1 < 800:
            rel.append("      ✓ Boca MODERADAMENTE aberta")
            rel.append("      ✓ Abertura natural, confortável")
        else:
            rel.append("      ✓ Boca BEM aberta, mandíbula baixa")
            rel.append("      ✓ Vogais mais abertas (como 'a', 'é')")

    rel.append("")

    # F2
    if len(analise_global['formantes']) > 1 and analise_global['formantes'][1]:
        f2 = analise_global['formantes'][1].get('F2_media', 0)
        rel.append(f"   F2 ({f2:.0f} Hz) - POSIÇÃO DA LÍNGUA:")
        if f2 < 1200:
            rel.append("      ✓ Língua para TRÁS, base elevada")
            rel.append("      ✓ Som mais 'escuro' e 'grosso'")
            rel.append("      ✓ Pense nas vogais 'o', 'u'")
        elif f2 < 2000:
            rel.append("      ✓ Língua em posição NEUTRA/MÉDIA")
            rel.append("      ✓ Som equilibrado")
        else:
            rel.append("      ✓ Língua para FRENTE, ponta elevada")
            rel.append("      ✓ Som mais 'claro' e 'brilhante'")
            rel.append("      ✓ Pense nas vogais 'i', 'e'")

    rel.append("")

    # F3-F5
    rel.append("   F3-F5 - RESSONÂNCIA GERAL:")
    envelope = analise_global['envelope_espectral']
    centroide = envelope['centroide_media']

    if centroide < 1500:
        rel.append("      ✓ Som ESCURO/AVELUDADO")
        rel.append("      ✓ Mais ressonância no PEITO")
        rel.append("      ✓ Direcione o som 'para baixo' e 'para trás'")
    elif centroide < 2500:
        rel.append("      ✓ Som EQUILIBRADO")
        rel.append("      ✓ Balance ressonância entre peito e cabeça")
    else:
        rel.append("      ✓ Som BRILHANTE/CLARO")
        rel.append("      ✓ Mais ressonância na MÁSCARA FACIAL")
        rel.append("      ✓ Direcione o som 'para frente' e 'para cima'")

    rel.append("")
    rel.append("💡 COMO PRATICAR:")
    rel.append("   1. Grave sua voz e compare com a original")
    rel.append("   2. Experimente diferentes posições de língua")
    rel.append("   3. Mude a abertura da boca")
    rel.append("   4. Use um espelho para observar")
    rel.append("")
    rel.append("")

    # 4. BREATHINESS
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 4. BREATHINESS - Quanto AR passa pela voz" + " " * 56 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    breathiness = analise_global['breathiness']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Nível: {breathiness['nivel'].upper()}")
    rel.append(f"   • Ratio: {breathiness['breathiness_ratio']:.3f}")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if breathiness['nivel'] == 'alto':
        rel.append("   ✓ VOZ COM MUITO AR (tipo Billie Eilish, Norah Jones)")
        rel.append("   ✓ Deixe as cordas vocais MENOS fechadas")
        rel.append("   ✓ Permita que MAIS ar escape enquanto canta")
        rel.append("   ✓ Som 'sussurrado', íntimo")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Não force o fechamento da garganta")
        rel.append("   2. Cante como se estivesse sussurrando")
        rel.append("   3. Ouça o 'shhh' do ar junto com a nota")
    elif breathiness['nivel'] == 'medio':
        rel.append("   ✓ VOZ EQUILIBRADA")
        rel.append("   ✓ Cordas vocais MODERADAMENTE fechadas")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Cante de forma natural e relaxada")
    else:
        rel.append("   ✓ VOZ LIMPA E COMPRIMIDA (tipo Whitney Houston, Adele)")
        rel.append("   ✓ Feche BEM as cordas vocais")
        rel.append("   ✓ Permita que POUCO ar escape")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Comprima as cordas vocais ao cantar")
        rel.append("   2. Som 'fechado', potente")
        rel.append("   3. Não deve ouvir 'ar' junto com a nota")

    rel.append("")
    rel.append("")

    # 5. BRILHO
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 5. BRILHO - Som claro/escuro" + " " * 69 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    brilho = analise_global['brilho']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Nível: {brilho['nivel'].upper()}")
    rel.append(f"   • Centróide: {brilho['brilho_hz']:.0f} Hz")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if brilho['nivel'] == 'brilhante':
        rel.append("   ✓ VOZ CLARA/BRILHANTE (muita energia nos agudos)")
        rel.append("   ✓ Direcione o som para a MÁSCARA FACIAL")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Imagine o som saindo pelo nariz/olhos")
        rel.append("   2. Sinta vibração na face (maçãs do rosto)")
        rel.append("   3. Som 'apontado', projetado")
    elif brilho['nivel'] == 'medio':
        rel.append("   ✓ VOZ EQUILIBRADA")
        rel.append("   ✓ Balance entre peito e cabeça")
    else:
        rel.append("   ✓ VOZ ESCURA/AVELUDADA (mais energia nos graves)")
        rel.append("   ✓ Direcione o som para o PEITO")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Imagine o som saindo do peito")
        rel.append("   2. Sinta vibração no peito (coloque a mão)")
        rel.append("   3. Som 'redondo', cheio")

    rel.append("")
    rel.append("")

    # 6. TENSÃO
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 6. TENSÃO VOCAL - Voz apertada ou relaxada" + " " * 55 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    tensao = analise_global['tensao']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Nível: {tensao['nivel'].upper()}")
    rel.append(f"   • Ratio: {tensao['tensao_ratio']:.3f}")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if tensao['nivel'] == 'alta':
        rel.append("   ✓ VOZ TENSA/COMPRIMIDA")
        rel.append("   ✓ Garganta mais FECHADA")
        rel.append("   ✓ Som 'apertado', intenso")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Comprima a garganta levemente")
        rel.append("   2. Som mais 'metal', incisivo")
        rel.append("   ⚠️  CUIDADO: Não force demais!")
    elif tensao['nivel'] == 'media':
        rel.append("   ✓ TENSÃO MODERADA")
        rel.append("   ✓ Equilíbrio entre tenso e relaxado")
    else:
        rel.append("   ✓ VOZ RELAXADA/ABERTA")
        rel.append("   ✓ Garganta BEM aberta")
        rel.append("   ✓ Som 'solto', natural")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Relaxe TODA a garganta")
        rel.append("   2. Como um bocejo: garganta aberta")
        rel.append("   3. Som 'fácil', fluido")

    rel.append("")
    rel.append("")

    # 7. ATAQUE
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 7. ATAQUE - Como começar cada nota" + " " * 62 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    attack = analise_global['adsr']['attack_time']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Attack Time: {attack:.4f} segundos")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if attack < 0.05:
        rel.append("   ✓ ATAQUE RÁPIDO/DURO")
        rel.append("   ✓ Notas começam IMEDIATAMENTE no volume máximo")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Comece a nota com força IMEDIATA")
        rel.append("   2. Como um 'golpe' vocal: PAM!")
    elif attack < 0.15:
        rel.append("   ✓ ATAQUE MODERADO")
        rel.append("   ✓ Notas começam rapidamente mas não abruptamente")
    else:
        rel.append("   ✓ ATAQUE SUAVE/LENTO")
        rel.append("   ✓ Notas começam GRADUALMENTE")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Comece a nota SUAVEMENTE")
        rel.append("   2. Vá CRESCENDO aos poucos")
        rel.append("   3. Como uma 'fade in' na voz")

    rel.append("")
    rel.append("")

    # 8. DINÂMICA
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 8. DINÂMICA - Variações de volume (forte/fraco)" + " " * 50 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    dinamica = analise_global['dinamica']
    range_din = dinamica['range_dinamico']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Range Dinâmico: {range_din:.1f} dB")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if range_din > 40:
        rel.append("   ✓ MUITA VARIAÇÃO de volume")
        rel.append("   ✓ Cante partes MUITO FORTES e partes MUITO FRACAS")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Identifique onde a pessoa canta FORTE")
        rel.append("   2. Identifique onde canta FRACO")
        rel.append("   3. Exagere o contraste")
    elif range_din > 25:
        rel.append("   ✓ VARIAÇÃO MODERADA")
        rel.append("   ✓ Algumas partes mais fortes, outras mais fracas")
    else:
        rel.append("   ✓ POUCA VARIAÇÃO (volume constante)")
        rel.append("   ✓ Mantenha volume UNIFORME")

    rel.append("")
    rel.append("")

    # 9. PORTAMENTOS
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 9. PORTAMENTOS - Deslizes entre notas" + " " * 60 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    portamentos = analise_global['portamentos']
    n_port = portamentos['portamentos_detectados']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • Portamentos: {n_port}")
    rel.append("")

    rel.append("🎯 O QUE FAZER:")

    if n_port > 20:
        rel.append("   ✓ MUITOS DESLIZES (estilo melismático)")
        rel.append("   ✓ Escorregue/deslize entre QUASE TODAS as notas")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. NÃO pule direto de uma nota para outra")
        rel.append("   2. DESLIZE suavemente")
        rel.append("   3. Como uma sirene: uuuuuuuuuu")
    elif n_port > 5:
        rel.append("   ✓ DESLIZES MODERADOS")
        rel.append("   ✓ Algumas conexões legato")
    else:
        rel.append("   ✓ POUCOS/SEM DESLIZES")
        rel.append("   ✓ Pule DIRETAMENTE de nota para nota")
        rel.append("")
        rel.append("💡 COMO FAZER:")
        rel.append("   1. Corte seco entre notas")
        rel.append("   2. Cada nota é separada: ta-ta-ta")

    rel.append("")
    rel.append("")

    # 10. QUALIDADE
    rel.append("┌" + "─" * 98 + "┐")
    rel.append("│ 10. QUALIDADE GERAL DA VOZ" + " " * 71 + "│")
    rel.append("└" + "─" * 98 + "┘")
    rel.append("")

    hnr = analise_global['hnr']

    rel.append(f"📊 DADOS TÉCNICOS:")
    rel.append(f"   • HNR: {hnr:.1f} dB")
    rel.append("")

    rel.append("🎯 RESULTADO:")
    if hnr > 15:
        rel.append("   ✓ Voz muito LIMPA e CLARA")
        rel.append("   ✓ Pouco ruído, som cristalino")
    elif hnr > 8:
        rel.append("   ✓ Voz EQUILIBRADA")
        rel.append("   ✓ Clareza natural")
    else:
        rel.append("   ✓ Voz mais ROUCA ou AÉREA")
        rel.append("   ✓ Mais ruído misturado com harmônicos")

    rel.append("")
    rel.append("=" * 100)

    return "\n".join(rel)

# ============================================================================
# FUNÇÃO PRINCIPAL
# ============================================================================
def transcrever_e_analisar(audio_path, model_size='medium', segmentacao_tempo=3.0):
    """Transcreve e analisa com segmentação controlada"""

    print(f"\n{'='*80}")
    print("SISTEMA DE ANÁLISE VOCAL COMPLETA")
    print(f"{'='*80}")

    # CARREGAR MODELO
    print(f"\n[1/5] Carregando modelo Whisper ({model_size})...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model(model_size, device=device)

    # DETECTAR IDIOMA
    print(f"\n[2/5] Detectando idioma...")
    audio_temp = whisper.load_audio(audio_path)
    audio_temp = whisper.pad_or_trim(audio_temp)
    mel = whisper.log_mel_spectrogram(audio_temp).to(model.device)
    _, probs = model.detect_language(mel)
    idioma = max(probs, key=probs.get)
    print(f"✓ Idioma: {idioma.upper()}")

    # TRANSCREVER
    print(f"\n[3/5] Transcrevendo áudio...")
    resultado = model.transcribe(
        audio_path,
        language=idioma,
        task='transcribe',
        verbose=False,
        word_timestamps=True
    )
    print(f"✓ {len(resultado['segments'])} segmentos detectados")

    # SEGMENTAR
    print(f"\n[4/5] Aplicando segmentação de {segmentacao_tempo}s...")
    segmentos_refinados = segmentar_por_tempo(resultado['segments'], segmentacao_tempo)
    print(f"✓ {len(segmentos_refinados)} segmentos após refinamento")

    # CARREGAR ÁUDIO
    print(f"\n[5/5] Analisando características vocais...")
    audio, sr = librosa.load(audio_path, sr=None)
    analisador = AnalisadorVocalCompleto()

    # ANÁLISE GLOBAL
    analise_global = analisador.analisar_segmento_completo(audio, sr, 0, len(audio)/sr)

    # GERAR RELATÓRIOS
    relatorio_pratico = gerar_relatorio_pratico(analise_global)

    # RELATÓRIO TÉCNICO
    relatorio_tecnico = []
    relatorio_tecnico.append("="*100)
    relatorio_tecnico.append("ANÁLISE TÉCNICA DETALHADA POR SEGMENTO")
    relatorio_tecnico.append("="*100)
    relatorio_tecnico.append(f"Segmentação: {segmentacao_tempo}s por segmento")
    relatorio_tecnico.append(f"Total de segmentos: {len(segmentos_refinados)}")
    relatorio_tecnico.append("="*100)
    relatorio_tecnico.append("")

    # ANALISAR CADA SEGMENTO
    for i, seg in enumerate(segmentos_refinados, 1):
        if i % 5 == 0:
            print(f"  Analisando segmento {i}/{len(segmentos_refinados)}...")

        analise_seg = analisador.analisar_segmento_completo(audio, sr, seg['start'], seg['end'])

        relatorio_tecnico.append(f"\n[{seg['start']:.2f}s - {seg['end']:.2f}s] {seg['text']}")
        relatorio_tecnico.append("-" * 100)

        # 1. PITCH (F0)
        if analise_seg.get('pitch'):
            p = analise_seg['pitch']
            f0_media_seg = p['media']

            # Classificação do tom
            if f0_media_seg < 150:
                tom_desc = "GRAVE"
            elif f0_media_seg < 250:
                tom_desc = "MÉDIO"
            elif f0_media_seg < 400:
                tom_desc = "AGUDO"
            else:
                tom_desc = "MUITO AGUDO"

            relatorio_tecnico.append(f"  1️⃣ PITCH: {p['media']:.1f} Hz ({tom_desc}) | Range: {p['min']:.1f}-{p['max']:.1f} Hz")

        # 2. VIBRATO
        if analise_seg.get('vibrato'):
            v = analise_seg['vibrato']
            if v.get('presente'):
                # Classificar velocidade
                if v['taxa_hz'] < 4:
                    vel_vib = "LENTO"
                elif v['taxa_hz'] < 6:
                    vel_vib = "MÉDIO"
                else:
                    vel_vib = "RÁPIDO"

                # Classificar intensidade
                if v['extensao_cents'] < 50:
                    int_vib = "sutil"
                elif v['extensao_cents'] < 100:
                    int_vib = "moderado"
                else:
                    int_vib = "intenso"

                relatorio_tecnico.append(f"  2️⃣ VIBRATO: {v['taxa_hz']:.1f} Hz ({vel_vib}), {v['extensao_cents']:.1f} cents ({int_vib})")
            else:
                relatorio_tecnico.append(f"  2️⃣ VIBRATO: Ausente ou muito sutil")

        # 3. FORMANTES (F1 e F2)
        if analise_seg.get('formantes'):
            formantes_seg = analise_seg['formantes']
            if len(formantes_seg) >= 2:
                f1_val = formantes_seg[0].get('F1_media', 0)
                f2_val = formantes_seg[1].get('F2_media', 0)

                if f1_val > 0 and f2_val > 0:
                    # Interpretação de F1 (abertura da boca)
                    if f1_val < 500:
                        f1_desc = "boca FECHADA"
                        f1_acao = "mandíbula relaxada"
                    elif f1_val < 800:
                        f1_desc = "boca MÉDIA"
                        f1_acao = "abertura natural"
                    else:
                        f1_desc = "boca ABERTA"
                        f1_acao = "mandíbula baixa"

                    # Interpretação de F2 (posição da língua)
                    if f2_val < 1200:
                        f2_desc = "língua TRÁS"
                        f2_acao = "som escuro"
                    elif f2_val < 2000:
                        f2_desc = "língua CENTRO"
                        f2_acao = "som equilibrado"
                    else:
                        f2_desc = "língua FRENTE"
                        f2_acao = "som brilhante"

                    relatorio_tecnico.append(f"  3️⃣ FORMANTES: F1={f1_val:.0f}Hz ({f1_desc}, {f1_acao}) | F2={f2_val:.0f}Hz ({f2_desc}, {f2_acao})")
                else:
                    relatorio_tecnico.append(f"  3️⃣ FORMANTES: Não detectados neste segmento")
            else:
                relatorio_tecnico.append(f"  3️⃣ FORMANTES: Dados insuficientes")
        else:
            relatorio_tecnico.append(f"  3️⃣ FORMANTES: Não analisados")

        # 4. BREATHINESS (AR NA VOZ)
        if analise_seg.get('breathiness'):
            b = analise_seg['breathiness']
            breath_nivel = b['nivel'].upper()
            breath_ratio = b['breathiness_ratio']

            if breath_nivel == 'ALTO':
                breath_acao = "deixe MUITO ar passar, cante sussurrado"
            elif breath_nivel == 'MEDIO':
                breath_acao = "ar moderado, natural"
            else:
                breath_acao = "POUCO ar, voz limpa e comprimida"

            relatorio_tecnico.append(f"  4️⃣ BREATHINESS: {breath_nivel} (ratio: {breath_ratio:.3f}) → {breath_acao}")

        # 5. BRILHO
        if analise_seg.get('brilho'):
            br = analise_seg['brilho']
            brilho_nivel = br['nivel'].upper()
            brilho_hz = br['brilho_hz']

            if brilho_nivel == 'BRILHANTE':
                brilho_acao = "som para FRENTE, máscara facial"
            elif brilho_nivel == 'MEDIO':
                brilho_acao = "som equilibrado"
            else:
                brilho_acao = "som para TRÁS, ressonância no peito"

            relatorio_tecnico.append(f"  5️⃣ BRILHO: {brilho_nivel} ({brilho_hz:.0f} Hz) → {brilho_acao}")

        # 6. TENSÃO VOCAL
        if analise_seg.get('tensao'):
            t = analise_seg['tensao']
            tensao_nivel = t['nivel'].upper()
            tensao_ratio = t['tensao_ratio']

            if tensao_nivel == 'ALTA':
                tensao_acao = "garganta FECHADA, som apertado"
            elif tensao_nivel == 'MEDIA':
                tensao_acao = "tensão equilibrada"
            else:
                tensao_acao = "garganta ABERTA, som relaxado"

            relatorio_tecnico.append(f"  6️⃣ TENSÃO: {tensao_nivel} (ratio: {tensao_ratio:.3f}) → {tensao_acao}")

        # 7. ATAQUE DAS NOTAS
        if analise_seg.get('adsr'):
            adsr = analise_seg['adsr']
            attack = adsr['attack_time']

            if attack < 0.05:
                attack_desc = "DURO/RÁPIDO"
                attack_acao = "comece a nota COM FORÇA imediata"
            elif attack < 0.15:
                attack_desc = "MODERADO"
                attack_acao = "comece firme mas não abrupto"
            else:
                attack_desc = "SUAVE/LENTO"
                attack_acao = "comece SUAVEMENTE, cresça aos poucos"

            relatorio_tecnico.append(f"  7️⃣ ATAQUE: {attack:.3f}s ({attack_desc}) → {attack_acao}")

        # 8. DINÂMICA (VOLUME)
        if analise_seg.get('dinamica'):
            d = analise_seg['dinamica']
            db_media = d['db_media']
            range_din = d['range_dinamico']

            # Classificar volume absoluto
            if db_media > -10:
                vol_desc = "MUITO FORTE"
            elif db_media > -20:
                vol_desc = "FORTE"
            elif db_media > -30:
                vol_desc = "MÉDIO"
            else:
                vol_desc = "FRACO"

            # Classificar variação
            if range_din > 40:
                var_desc = "MUITA variação"
            elif range_din > 25:
                var_desc = "variação moderada"
            else:
                var_desc = "volume constante"

            relatorio_tecnico.append(f"  8️⃣ DINÂMICA: {db_media:.1f} dB ({vol_desc}) | Range: {range_din:.1f} dB ({var_desc})")

        # 9. PORTAMENTOS (DESLIZES)
        if analise_seg.get('portamentos'):
            port = analise_seg['portamentos']
            num_port = port['portamentos_detectados']

            if num_port > 5:
                port_desc = "MUITOS deslizes"
                port_acao = "escorregue entre as notas (glissando)"
            elif num_port > 2:
                port_desc = "alguns deslizes"
                port_acao = "deslize ocasionalmente"
            else:
                port_desc = "POUCOS/SEM deslizes"
                port_acao = "pule DIRETO de nota para nota"

            relatorio_tecnico.append(f"  9️⃣ PORTAMENTOS: {num_port} detectados ({port_desc}) → {port_acao}")

        # 10. QUALIDADE GERAL (HNR)
        if analise_seg.get('hnr') is not None:
            hnr = analise_seg['hnr']

            if hnr > 15:
                hnr_desc = "voz MUITO LIMPA"
                hnr_acao = "som cristalino, pouco ruído"
            elif hnr > 8:
                hnr_desc = "voz EQUILIBRADA"
                hnr_acao = "clareza natural"
            else:
                hnr_desc = "voz ROUCA/AÉREA"
                hnr_acao = "mais ruído misturado"

            relatorio_tecnico.append(f"  🔟 QUALIDADE: HNR {hnr:.1f} dB ({hnr_desc}) → {hnr_acao}")

        # RESSONÂNCIA (BÔNUS)
        if analise_seg.get('ressonancia'):
            r = analise_seg['ressonancia']
            res_tipo = r['tipo'].upper()
            res_freq = r['freq_dominante']

            res_descricoes = {
                'PEITORAL': 'grave, vibração no peito',
                'MISTA': 'transição entre registros',
                'CABECA': 'agudo, ressonância na cabeça',
                'NASAL': 'ressonância nasal',
                'FALSETE': 'leve e aéreo'
            }

            res_desc = res_descricoes.get(res_tipo, 'normal')
            relatorio_tecnico.append(f"  ➕ RESSONÂNCIA: {res_tipo} ({res_freq:.0f} Hz) - {res_desc}")

        # Linha em branco para separação
        relatorio_tecnico.append("")

    print(f"✓ Análise completa!")

    # COMBINAR RELATÓRIOS
    relatorio_final = relatorio_pratico + "\n\n\n" + "\n".join(relatorio_tecnico)

    return relatorio_final, analise_global

print("✓ Funções definidas!")

# ============================================================================
# EXECUTAR ANÁLISE
# ============================================================================
print("\n" + "="*80)
print("EXECUTANDO ANÁLISE VOCAL COMPLETA")
print("="*80)

try:
    if not os.path.exists(AUDIO_PATH):
        print(f"\n❌ ERRO: Arquivo não encontrado: {AUDIO_PATH}")
    else:
        resultado_final, dados_globais = transcrever_e_analisar(
            AUDIO_PATH,
            MODEL_SIZE,
            SEGMENTACAO_TEMPO
        )

        # SALVAR
        output_path = AUDIO_PATH.replace('.wav', '_analise_pratica.txt')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(resultado_final)

        print(f"\n{'='*80}")
        print(f"✓ Arquivo salvo: {output_path}")
        print(f"{'='*80}")

        # MOSTRAR PRÉVIA
        print("\n" + "="*80)
        print("PRÉVIA DO RELATÓRIO PRÁTICO:")
        print("="*80)
        print(resultado_final[:2000] + "\n...\n(arquivo completo salvo)")

        print("\n✅ ANÁLISE COMPLETA FINALIZADA COM SUCESSO!")

except Exception as e:
    print(f"\n❌ ERRO: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("📥 Arquivo gerado está no seu Google Drive!")
print("="*80)

📦 Instalando dependências...
✅ Ambiente pronto para análise vocal!
✓ Bibliotecas importadas!

📁 Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Drive montado!

⚙️ Configurações:
✓ Arquivo: /content/drive/MyDrive/AudioAnalysis/AmyWinehouse/Back_to_Black_vocals.wav
✓ Modelo Whisper: medium
✓ Segmentação de tempo: 3.0s (cada segmento será analisado separadamente)
✓ Arquivo encontrado!
✓ Tamanho: 40.38 MB
✓ Classes definidas!
✓ Funções definidas!

EXECUTANDO ANÁLISE VOCAL COMPLETA

SISTEMA DE ANÁLISE VOCAL COMPLETA

[1/5] Carregando modelo Whisper (medium)...

[2/5] Detectando idioma...
✓ Idioma: EN

[3/5] Transcrevendo áudio...


100%|█████████▉| 23909/24004 [12:31<00:02, 31.82frames/s]


✓ 16 segmentos detectados

[4/5] Aplicando segmentação de 3.0s...
✓ 77 segmentos após refinamento

[5/5] Analisando características vocais...
  Analisando segmento 5/77...
  Analisando segmento 10/77...
  Analisando segmento 15/77...
  Analisando segmento 20/77...
  Analisando segmento 25/77...
  Analisando segmento 30/77...
  Analisando segmento 35/77...
  Analisando segmento 40/77...
  Analisando segmento 45/77...
  Analisando segmento 50/77...
  Analisando segmento 55/77...
  Analisando segmento 60/77...
  Analisando segmento 65/77...
  Analisando segmento 70/77...
  Analisando segmento 75/77...
✓ Análise completa!

✓ Arquivo salvo: /content/drive/MyDrive/AudioAnalysis/AmyWinehouse/Back_to_Black_vocals_analise_pratica.txt

PRÉVIA DO RELATÓRIO PRÁTICO:
GUIA PRÁTICO DE REPLICAÇÃO VOCAL - O QUE FAZER PARA IMITAR ESTA VOZ

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ 1. ALTURA DA VOZ (PITCH/TOM) - Qual nota cantar                